## 01_simulation.ipynb

This notebook demonstrates the **Simulation** capabilities of the `pysubmit` package. Specifically, it walks through:

1. Running an **Eigenmode** simulation
2. Inspecting and saving simulation results
3. Serialize and save results
4. Adjusting simulation parameters
5. Serializing and deserializing simulation objects

> **Note**:
> The `pysubmit.simulation` module supports multiple simulation classes.  
> In this tutorial, we focus on two: `EigenmodeAnalysis`.


In [1]:
# 1.1 Import required classes
from pysubmit.simulation import EigenmodeAnalysis
from pyaedt.hfss import Hfss

### Step 1: Set up and run an Eigenmode simulation

We’ll begin by opening an existing HFSS project using `pyaedt`, then use it to create and run an eigenmode simulation.

In [2]:
# 1.2 Open an existing HFSS project and run the simulation
with Hfss(
    project='simple_design_readout.aedt',
    design='my_design',
    remove_lock=True,
    close_on_exit=True
) as hfss:
    # Create the simulation object
    simulation = EigenmodeAnalysis(setup_name='Setup1', design_name='my_design')

    # Run the simulation and store the result
    result = simulation.analyze(hfss)


PyAEDT INFO: Parsing simple_design_readout.aedt.
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)].
PyAEDT INFO: PyAEDT version 0.15.1.
PyAEDT INFO: File simple_design_readout.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\rosengrp\AppData\Local\Temp\pyaedt_rosengrp_9e1047e2-1f1d-4efe-992b-80922eb6f84f.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 52477.
PyAEDT INFO: Electronics Desktop started on gRPC port: 52477 after 17.887817859649658 seconds.
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 34380.
PyAEDT INFO: Project simple_d

### Step 2: Inspect the simulation results

Once the simulation is complete, it returns a result object.  
The structure of this object dependss on the simulation type, but all result objects share two important features:

- They are **printable** for easy human inspection
- They are **JSON serializable** for saving and reuse


In [3]:
# 2.1 Print the result summary
print("Eigenmode Simulation Results:")
print(result)

# 2.2 Access specific data, such as mode 1 frequency and Q-factor
print(f"Mode 1 frequency: {result.results[1].frequency}")
print(f"Mode 1 Q-factor: {result.results[1].quality_factor}")


Eigenmode Simulation Results:
id='' type=<SimulationOutputTypesNames.EIGENMODE_RESULT: 'eigenmode_result'> results={1: SingleModeResult(mode_number=1, quality_factor=0.0, frequency=Value(value=7.96568506802845, unit='GHz'), label=None)} frequencies_unit='GHz'
Mode 1 frequency: value=7.96568506802845 unit='GHz'
Mode 1 Q-factor: 0.0


### Step 3: Serialize and save results

You can serialize the results into JSON format for logging or further processing, and save them to a file.

In [4]:
# 3.1 Serialize the result to a JSON string
result_as_json = result.model_dump_json(indent=4)

# 3.2 Print the serialized result
print("Serialized Results:")
print(result_as_json)

# 3.3 Save the result to a file
with open('my_results.json', 'w') as f:
    f.write(result_as_json)

Serialized Results:
{
    "id": "",
    "type": "eigenmode_result",
    "results": {
        "1": {
            "mode_number": 1,
            "quality_factor": 0.0,
            "frequency": {
                "value": 7.96568506802845,
                "unit": "GHz"
            },
            "label": null
        }
    },
    "frequencies_unit": "GHz"
}


### Step 4: Adjust simulation setup parameters

Before running the simulation, you can also modify the simulation setup parameters to suit your needs.

> **Tip:**  
> For a full list of configurable parameters for each setup type, refer to the official  
> [Ansys AEDT Setup Templates documentation](https://aedt.docs.pyansys.com/version/stable/API/SetupTemplates.html)

In [5]:
# 4.1 Customize the simulation parameters
setup_parameters = {
    'MaximumPasses': 4,
    'MinimumConvergedPasses': 1,
    'MinimumPasses': 1,
    'MinimumFrequency': '2GHz',
    'NumModes': 3,
    'MaxDeltaFreq': 0.2,
    'ConvergeOnRealFreq': True,
    'PercentRefinement': 30,
    'BasisOrder': -1,
    'DoLambdaRefine': True,
    'DoMaterialLambda': True,
    'SetLambdaTarget': False,
    'Target': 0.4,
    'UseMaxTetIncrease': False,
}

simulation.setup_parameters = setup_parameters
print(f'Simulation with updated parameters: \n{simulation}')
# Re-run with: simulation.analyze(hfss) if desired


Simulation with updated parameters: 
id='' type=<SimulationTypesNames.EIGENMODE: 'eigenmode'> setup_name='Setup1' design_name='my_design' cores=4 gpus=0 setup_parameters={'MaximumPasses': 4, 'MinimumConvergedPasses': 1, 'MinimumPasses': 1, 'MinimumFrequency': '2GHz', 'NumModes': 3, 'MaxDeltaFreq': 0.2, 'ConvergeOnRealFreq': True, 'PercentRefinement': 30, 'BasisOrder': -1, 'DoLambdaRefine': True, 'DoMaterialLambda': True, 'SetLambdaTarget': False, 'Target': 0.4, 'UseMaxTetIncrease': False}


### Step 5: Serialize and reload the simulation object

In some workflows, you may want to reuse the same simulation configuration across different designs or sessions.

To support this, you can serialize the entire simulation object to JSON and reload it when needed.


In [6]:
# 5.1 Serialize the entire simulation object
simulation_as_string = simulation.model_dump_json(indent=4)
print("Serialized Simulation Object:")
print(simulation_as_string)

# 5.2 Save the simulation to disk
with open('my_simulation.json', 'w') as f:
    f.write(simulation_as_string)

# 5.3 Load it from disk
with open('my_simulation.json', 'r') as f:
    data = f.read()

# 5.4 Reconstruct the simulation object
simulation_loaded = EigenmodeAnalysis.model_validate_json(data)
print("Loaded Simulation Object:")
print(simulation_loaded)


Serialized Simulation Object:
{
    "id": "",
    "type": "eigenmode",
    "setup_name": "Setup1",
    "design_name": "my_design",
    "cores": 4,
    "gpus": 0,
    "setup_parameters": {
        "MaximumPasses": 4,
        "MinimumConvergedPasses": 1,
        "MinimumPasses": 1,
        "MinimumFrequency": "2GHz",
        "NumModes": 3,
        "MaxDeltaFreq": 0.2,
        "ConvergeOnRealFreq": true,
        "PercentRefinement": 30,
        "BasisOrder": -1,
        "DoLambdaRefine": true,
        "DoMaterialLambda": true,
        "SetLambdaTarget": false,
        "Target": 0.4,
        "UseMaxTetIncrease": false
    }
}
Loaded Simulation Object:
id='' type=<SimulationTypesNames.EIGENMODE: 'eigenmode'> setup_name='Setup1' design_name='my_design' cores=4 gpus=0 setup_parameters={'MaximumPasses': 4, 'MinimumConvergedPasses': 1, 'MinimumPasses': 1, 'MinimumFrequency': '2GHz', 'NumModes': 3, 'MaxDeltaFreq': 0.2, 'ConvergeOnRealFreq': True, 'PercentRefinement': 30, 'BasisOrder': -1, 'DoLam